# Clustering ensemble

Grupowanie może być także używane jako preprocessing do klasyfikacji. Idea jest zbliżona do komitetów klasyfikatorów.

ZADANIE: Proszę wziąć dane ecoli z UCI. Podzielić na zbiór testowy i treningowy.

1. Uczymy logistyczną regresję i testujemy (accuracy)

2. Na zbiorze treningowym robimy k-means (5 klastrów) co daje podział. W każdym klastrze uczymy logistyczną regresję. Testujemy przypisując punkt do najbliższego klastra, a następnie przewidując modelem klasyfikacyjnym z tego klastra.

Które podejście daje lepszy wynik. Wyjaśnij zalety drugiego podejścia na przykładzie 2D.

In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv(
    filepath_or_buffer='data_cl/ecoli.dat', 
    header=None, 
    sep=',')

X = np.array(df.iloc[:, :-1])
X
print(X.shape)
y = df.iloc[:,-1]

labels = set(y)
labels
#{cp,im,imS,imL,imU,om,omL,pp}

(336, 7)


{'cp', 'im', 'imL', 'imS', 'imU', 'om', 'omL', 'pp'}

In [2]:
z = {'cp':0, 'im':1, 'imL':2, 'imS':3, 'imU':4, 'om':5, 'omL':6, 'pp':7}
z

{'cp': 0, 'im': 1, 'imL': 2, 'imS': 3, 'imU': 4, 'om': 5, 'omL': 6, 'pp': 7}

In [3]:
Y = np.array([z[key] for key in y])
Y

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       3, 3, 2, 2, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 7, 7,
       7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7,

In [5]:
from sklearn import linear_model, datasets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,Y, test_size = 0.2)
print(X_train.shape)
print(X_test.shape)

(268, 7)
(68, 7)


In [10]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_s = scaler.fit_transform(X_train)
X_test_s = scaler.transform(X_test)

In [11]:
logreg = linear_model.LogisticRegression(C=1e5)
logreg.fit(X_train_s, y_train)

LogisticRegression(C=100000.0, class_weight=None, dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)

In [12]:
from sklearn.metrics import accuracy_score

y_train_pred = logreg.predict(X_train_s)
y_test_pred = logreg.predict(X_test_s)

print("Trening ", accuracy_score(y_train, y_train_pred))
print("Test ", accuracy_score(y_test, y_test_pred))

Trening  0.8171641791044776
Test  0.8235294117647058


In [42]:
from sklearn.cluster import KMeans

k=5
cluster = KMeans(n_clusters = k, random_state = 42)
clust_train = cluster.fit_predict(X_train_s)

clust_test = cluster.predict(X_test_s)

In [43]:
def split(X,y, clustering):
    data = []
    labels = []
    for j in range(len(set(clustering))):
        data.append(X[clustering == j])
        labels.append(y[clustering == j])
    return (data, labels)

In [44]:
data_train, labels_train = split(X_train, y_train, clust_train)
data_test, labels_test = split(X_test, y_test, clust_test)

In [45]:
acc_train = 0
acc_test = 0

logreg = linear_model.LogisticRegression(C = 1e5, random_state = 42)
for i in range(len(set(clust_train))):
    logreg.fit(data_train[i], labels_train[i])
    y_train_pred = logreg.predict(data_train[i])
    acc_tmp = accuracy_score(labels_train[i], y_train_pred)
    print("acc train ", acc_tmp)
    acc_train += acc_tmp * len(labels_train[i])
    
    
    y_test_pred = logreg.predict(data_test[i])
    acc_tmp = accuracy_score(labels_test[i], y_test_pred)
    print("acc test ", acc_tmp)
    acc_test += acc_tmp * len(labels_test[i])
    
acc_train = acc_train/len(X_train)    
acc_test = acc_test/len(X_test)    
    
print("train ", acc_train)
print("test ", acc_test)

acc train  1.0
acc test  0.5
acc train  0.8870967741935484
acc test  0.9032258064516129
acc train  0.8028169014084507
acc test  0.6153846153846154


ValueError: This solver needs samples of at least 2 classes in the data, but the data contains only one class: 2